# BLOCKY Zipperchain Demo

At BLOCKY, we help you trust your data by allowing you to validate the integrity of your files. With BLOCKY, you can sign files onto Zipperchain, our custom blockchain, and retrieve certificates of signed data.

This demo introduces you to the basics of Zipperchain.

In this demo we will be:

* Setting up our Collaboratory environment.
* Creating data to sign onto Zipperchain.
* Signing data onto Zipperchain.
* Fetching the receipt of our data that locates our signed data on Zipperchain. 
* Fetching the certificate that document the state of our signed data.
* Verifying that the certificate matches our local data.

This demo uses the testnet version of Zipperchain so be mindful that anything recorded here will likely be deleted at some point!

You will notice we are using Colaboratory, a free Jupyter notebook environment provided by Google, for this demonstration. If this is your first time playing with a Colaboratory/Jupyter notebook, here are some tips:

* To run the code cells in this procedure, click on it and then click on the <img src="https://blocky-public.s3.amazonaws.com/play_dark.png" alt="Play button." width="18"/> button. 
* Python is Collaboratory's language of choice, so that is what we are using here.
* The `!` triggers a shell commands within a Colaboratory notebook.


# Setting Up

To begin, let's install a few Python libraries to support us in our journey.

In [ ]:
! pip install requests

In [ ]:
# import libraries
import uuid
import hashlib
import requests
import pprint

# Creating Data

In order to sign some data, we need to create some data. Zipperchain can store almost any kind of data, from text files to database backups. For this tutorial, we'll create a text file for us to use and take a look at the contents with the `cat` command.

In [ ]:
# create data
! echo "It's $(date) and I am awesome!" > example.txt
! cat example.txt

Throughout this demo we will be using this data in our interactions with Zipperchain.



# Signing Data

Now let's write a signature of the data onto the Zipperchain blockchain. Signing data requires two inputs: a user ID and our data. So first, let's generate a unique user ID for our session.

In [ ]:
# create user id
signer_id = str(uuid.uuid4())
print(signer_id)

Next, becuase we want to verify the integrity of our local data, let's generate a hash of our file and take a look at it.

In [ ]:
# create hash
with open("example.txt") as f:
  data = f.read().encode('ascii', 'replace')
  content_hash = hashlib.sha256(data).hexdigest()
print(content_hash)

Now we have everything we need to write to Zipperchain. To do this, we are going to combine our user ID and hash into a payload which we deliver to Zipperchain as a REST API POST request. After this runs, let's check the response to be sure the request was accepted. 

In [ ]:
# sign data
payload = {'signer_id': signer_id, 'data': content_hash}
response = requests.post('http://demo.zpr.bky.sh:3001/receipt', json=payload)
print(response)

Great, now we have data signed onto Zipperchain!

What just happened?
* The call took in our `payload` and sent it to the Zipperchain API.
* The server recorded the request in its database and then sent the data to the Zipperchain blockchain.
* The server recorded a certificate of the file signature (containing our user ID and the hash of `example.txt`).
* We know the submission was successful by the `202` response from the server.

That means that Zipperchain has now recorded the user ID and hash! Now we are ready to fetch our entry.


# Fetching the Receipt

In order to verify our data, first we need the receipt of our entry from Zipperchain so we can locate the signature.  This takes in your `userID` and `Data` and returns a `receipt` of the entry. Once we process this request, let's take a look at its contents.

In [ ]:
# fetch receipt
response = requests.get(f'http://demo.zpr.bky.sh:3001/receipt?signer_id={signer_id}&data={content_hash}')
receipt = response.json()[0]
pprint.pprint(receipt)

The request has returned a JSON receipt of our Zipperchain entry with 4 fields:    
* **block_num:** the block number of the Zipperchain entry
* **branch:** the location of our entry within the block
* **signature:** the data that we included in our signature
* **zigzag_id:** the...

With this receipt we can now request the certificate of our data, which pulls our entry directly from Zipperchain and can be used to attest to the integrity of our local file.


# Fetching the Certificate

Now let's fetch our certificate which attests to the data we submitted to Zipperchain. To do this, we will send a POST request to Zipperchain which includes our `receipt`. Once this request is complete, let's look at the response.

In [ ]:
# fetch certificate
response = requests.post('http://demo.zpr.bky.sh:3001/certificate', json=receipt)
print(response)
certificate = response.json()
pprint.pprint(certificate)

The request returned another JSON with our certificate data which includes 3 fields:    
* **lower_bound:** this is the timestamp of the Attestation (link to Zipper white paper) which preceded the block where our data is stored
* **receipt:** this is the data from Zipperchain which describes the block, block location, signature data, and Zigzag ID of our data
* **upper_bound:** this is the timestamp of the attestation (link to Zipper white paper) which followed our block

This certificate is the Zipperchain source of truth for the data we entered.  We can see our user ID and data hash we included in this entry by looking at the `signature` field of the response.

In [ ]:
# signed data
print(certificate['receipt']['signature'])

# Verifying Data

To conclude this demo, we want to make sure our local data matches the certificate. Let's isolate the hash of our data in the certificate (`zipper_sig`) and compare that to the hash of the data we created earlier in this demo (`content_hash`).

In [ ]:
zipper_sig = certificate['receipt']['signature']['data']
print(zipper_sig == content_hash)

Hurray! Our local data and the entry on Zipperchain match! 

This concludes the demo for Zipperchain. Thanks for taking a look and feel free to reach out to taylor@blocky.rocks with any questions or comments.